In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import matplotlib.pyplot as plt
import lsst.afw.table

from astropy.io import fits
from astropy.table import Table, hstack, vstack
import sys
sys.path.append('/home/jiaxuanl/Research/Merian/merian_tractor/scripts/')

In [2]:
import os
old_patches = [name for name in os.listdir(
    "/projects/MERIAN/repo/S20A/deepCoadd_calexp/9813/")]
new_patches = [int(name[0]) + int(name[2]) * 9 for name in old_patches]
merian_patches = [int(name) for name in os.listdir(
    "/projects/MERIAN/repo/DECam/runs/merian/dr1_wide/20220921T193246Z/deepCoadd_forced_src/9813")]
common_patches = np.intersect1d(new_patches, merian_patches)

In [3]:
def rsync(new_patches, filt):
    old_patches = f'{new_patches % 9},{new_patches // 9}'
    return f'rsync -azv --progress jiaxuanl@tiger.princeton.edu://projects/MERIAN/repo/S20A/gaapTable/9813/{old_patches}/gaapTable_{filt}_9813_{old_patches}.fits ./'

In [4]:
rsync(25, 'I')

'rsync -azv --progress jiaxuanl@tiger.princeton.edu://projects/MERIAN/repo/S20A/gaapTable/9813/7,2/gaapTable_I_9813_7,2.fits ./'

In [8]:
def joinCatBands(patch=23, filters='griz', tract=9813):
    cats = []
    patch_old = f'{patch % 9},{patch // 9}'
    for i, filt in enumerate(list(filters)):
        temp = Table.read(f'/projects/MERIAN/repo/S20A/gaapTable/{tract}/{patch_old}/gaapTable_{filt.upper()}_{tract}_{patch_old}.fits')
        if i > 0:
            temp.remove_columns(['id', 'coord_ra', 'coord_dec'])
        cats.append(temp)
    return hstack(cats)

def joinCatPatches(patches, filters='griz', tract=9813):
    cats = []
    for i in patches:
        cats.append(joinCatBands(patch=i, filters=filters, tract=tract))
    return vstack(cats)

def joinMerianCatPatches(patches, tract=9813):
    import lsst.daf.butler as dafButler
    butler = dafButler.Butler('/projects/MERIAN/repo/')
    
    cats = []
    for patch in patches:
        patch_old = f'{patch % 9},{patch // 9}'
        dataId = dict(tract=tract, patch=patch)
        refCat = butler.get(
            'objectTable',
            collections='DECam/runs/merian/dr1_wide',
            dataId=dataId,
            instrument='DECam',
            skymap='hsc_rings_v1',
        )
        refCat = Table.from_pandas(refCat, index=True)
        cats.append(refCat)
    return vstack(cats)

In [9]:
HscCat = joinCatPatches([23, 24, 25], filters='gri')
MerianCat = joinMerianCatPatches([23, 24, 25])

In [10]:
len(HscCat)

102914

rsync -azv --progress jiaxuanl@tiger.princeton.edu://projects/MERIAN/repo/S20A/gaapTable/9813/gaap_9813_23_25_gri.fits ./

In [11]:
HscCat.remove_columns(['coord_ra', 'coord_dec'])
HscCat.rename_column('id', 'hsc_id')
bigCat = hstack([MerianCat, HscCat])

In [13]:
bigCat.write('/projects/MERIAN/repo/S20A/gaapTable/9813/gaap_9813_23_25_gri.fits')

rsync -azv --progress jiaxuanl@tiger.princeton.edu://projects/MERIAN/repo/S20A/gaapTable/9813/gaap_9813_23_25_refN540.fits ./